# Course Data Cleaning Pipeline

This notebook performs comprehensive data cleaning on the Coursera courses dataset:

1. **Field Normalization**: Clean and standardize title, description, provider, url, price, is_free, level, hours, rating, language, tags
2. **Deduplication**: Remove duplicates based on (lowercased) title + provider
3. **Dead Link Removal**: Identify and remove courses with invalid URLs
4. **Canonical Tagging**: Add standardized tags based on course content
5. **Export**: Save cleaned data as parquet file

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from urllib.parse import urlparse
import ast
from typing import List, Dict, Set
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


## 1. Load and Inspect Data

In [2]:
# Load the original dataset
df = pd.read_csv('../Data/coursera_courses.csv')
print(f"Original dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()

Original dataset shape: (1000, 12)
Columns: ['course_title', 'course_organization', 'course_certificate_type', 'course_time', 'course_rating', 'course_reviews_num', 'course_difficulty', 'course_url', 'course_students_enrolled', 'course_skills', 'course_summary', 'course_description']


,course_title,course_organization,course_certificate_type,course_time,course_rating,course_reviews_num,course_difficulty,course_url,course_students_enrolled,course_skills,course_summary,course_description
0,(ISC)² Systems Security Certified Practitioner...,ISC2,Specialization,3 - 6 Months,4.7,492,Beginner,https://www.coursera.org/specializations/sscp-...,"6,958","['Risk Management', 'Access Control', 'Asset',...",[],Pursue better IT security job opportunities an...
1,.NET FullStack Developer,Board Infinity,Specialization,1 - 3 Months,4.3,51,Intermediate,https://www.coursera.org/specializations/dot-n...,"2,531","['Web API', 'Web Development', 'Cascading Styl...",['Master .NET full stack web dev: from .NET co...,Develop the proficiency required to design and...
2,21st Century Energy Transition: how do we make...,University of Alberta,Course,1 - 3 Months,4.8,62,Beginner,https://www.coursera.org/learn/21st-century-en...,"4,377",[],['Understand the complexity of systems supplyi...,"Affordable, abundant and reliable energy is fu..."
3,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,Course,1 - 3 Months,4.7,517,Intermediate,https://www.coursera.org/learn/crash-course-in...,"39,004","['Instrumental Variable', 'Propensity Score Ma...",[],We have all heard the phrase “correlation does...
4,A life with ADHD,University of Geneva,Course,1 - 3 Months,NaN,NaN,Beginner,https://www.coursera.org/learn/life-with-adhd,NaN,"['differential diagnosis and comorbidities', '...",[' Understand what ADHD is and the challenges ...,What is ADHD and what are the challenges that ...


In [3]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())
print("\nData types:")
print(df.dtypes)

Missing values:
course_title                 0
course_organization          0
course_certificate_type      0
course_time                  0
course_rating                6
course_reviews_num           6
course_difficulty            0
course_url                   0
course_students_enrolled    41
course_skills                0
course_summary               0
course_description           1
dtype: int64

Data types:
course_title                 object
course_organization          object
course_certificate_type      object
course_time                  object
course_rating               float64
course_reviews_num           object
course_difficulty            object
course_url                   object
course_students_enrolled     object
course_skills                object
course_summary               object
course_description           object
dtype: object


## 2. Field Normalization

In [4]:
# Create a copy for cleaning
courses_clean = df.copy()

# Rename columns to match requested schema
column_mapping = {
    'course_title': 'title',
    'course_organization': 'provider', 
    'course_url': 'url',
    'course_rating': 'rating',
    'course_difficulty': 'level',
    'course_time': 'hours',
    'course_description': 'description',
    'course_skills': 'tags',
    'course_certificate_type': 'certificate_type',
    'course_reviews_num': 'reviews_count',
    'course_students_enrolled': 'students_enrolled',
    'course_summary': 'summary'
}

courses_clean = courses_clean.rename(columns=column_mapping)
print(f"Renamed columns: {list(courses_clean.columns)}")

Renamed columns: ['title', 'provider', 'certificate_type', 'hours', 'rating', 'reviews_count', 'level', 'url', 'students_enrolled', 'tags', 'summary', 'description']


In [5]:
def normalize_text(text):
    """Normalize text fields: lowercase, strip whitespace, handle NaN"""
    if pd.isna(text):
        return ''
    return str(text).lower().strip()

def clean_title(title):
    """Clean and normalize course titles"""
    if pd.isna(title):
        return ''
    
    title = str(title).strip()
    # Remove extra whitespace
    title = re.sub(r'\s+', ' ', title)
    # Remove special characters at the start/end
    title = re.sub(r'^[^a-zA-Z0-9]+|[^a-zA-Z0-9]+$', '', title)
    
    return title.lower()

def clean_provider(provider):
    """Clean and normalize provider names"""
    if pd.isna(provider):
        return ''
    
    provider = str(provider).strip().lower()
    # Standardize university names
    provider = re.sub(r'\buniversity\b', 'university', provider)
    provider = re.sub(r'\bcollege\b', 'college', provider)
    
    return provider

def validate_url(url):
    """Validate URL format"""
    if pd.isna(url):
        return False
    
    try:
        result = urlparse(str(url))
        return all([result.scheme, result.netloc])
    except:
        return False

def extract_hours(time_str):
    """Extract numeric hours from time string"""
    if pd.isna(time_str):
        return np.nan
    
    time_str = str(time_str).lower()
    
    # Handle different time formats
    if 'month' in time_str:
        # Convert months to hours (assuming 4 weeks per month, 10 hours per week)
        months = re.findall(r'(\d+)\s*-?\s*(\d+)?\s*month', time_str)
        if months:
            start_months = int(months[0][0])
            end_months = int(months[0][1]) if months[0][1] else start_months
            avg_months = (start_months + end_months) / 2
            return avg_months * 4 * 10  # Convert to hours
    
    elif 'week' in time_str:
        weeks = re.findall(r'(\d+)\s*-?\s*(\d+)?\s*week', time_str)
        if weeks:
            start_weeks = int(weeks[0][0])
            end_weeks = int(weeks[0][1]) if weeks[0][1] else start_weeks
            avg_weeks = (start_weeks + end_weeks) / 2
            return avg_weeks * 10  # Assuming 10 hours per week
    
    elif 'hour' in time_str:
        hours = re.findall(r'(\d+)\s*-?\s*(\d+)?\s*hour', time_str)
        if hours:
            start_hours = int(hours[0][0])
            end_hours = int(hours[0][1]) if hours[0][1] else start_hours
            return (start_hours + end_hours) / 2
    
    return np.nan

def normalize_level(level):
    """Normalize difficulty levels"""
    if pd.isna(level):
        return 'unknown'
    
    level = str(level).lower().strip()
    
    if level in ['beginner', 'introductory', 'intro', 'basic']:
        return 'beginner'
    elif level in ['intermediate', 'medium']:
        return 'intermediate'
    elif level in ['advanced', 'expert']:
        return 'advanced'
    else:
        return 'unknown'

print("Normalization functions defined")

Normalization functions defined


In [6]:
# Apply normalization
print("Normalizing fields...")

# Normalize text fields
courses_clean['title'] = courses_clean['title'].apply(clean_title)
courses_clean['description'] = courses_clean['description'].apply(normalize_text)
courses_clean['provider'] = courses_clean['provider'].apply(clean_provider)
courses_clean['level'] = courses_clean['level'].apply(normalize_level)

# Extract hours from time field
courses_clean['hours'] = courses_clean['hours'].apply(extract_hours)

# Add missing fields with defaults
courses_clean['price'] = 0  # Assuming all courses are free for now
courses_clean['is_free'] = True
courses_clean['language'] = 'english'  # Default assumption

# Validate URLs
courses_clean['valid_url'] = courses_clean['url'].apply(validate_url)

print(f"After normalization: {courses_clean.shape}")
print(f"Invalid URLs: {(~courses_clean['valid_url']).sum()}")

Normalizing fields...
After normalization: (1000, 16)
Invalid URLs: 0


## 3. Deduplication

In [7]:
# Create deduplication key
courses_clean['dedup_key'] = courses_clean['title'] + '|||' + courses_clean['provider']

print(f"Before deduplication: {len(courses_clean)}")
print(f"Unique title+provider combinations: {courses_clean['dedup_key'].nunique()}")
print(f"Duplicates to remove: {len(courses_clean) - courses_clean['dedup_key'].nunique()}")

# Check some duplicates
duplicates = courses_clean[courses_clean.duplicated('dedup_key', keep=False)]
if len(duplicates) > 0:
    print("\nSample duplicates:")
    print(duplicates[['title', 'provider', 'rating', 'reviews_count']].head(10))

Before deduplication: 1000
Unique title+provider combinations: 991
Duplicates to remove: 9

Sample duplicates:
                         title provider  rating reviews_count
427      google data analytics   google     4.8         15.2k
428      google data analytics   google     4.8          132k
434          google it support   google     4.7         41.8k
435          google it support   google     4.8        180.7k
439  google project management   google     4.7          2.8k
442  google project management   google     4.9          1.8k
443           google ux design   google     4.8         96.3k
444           google ux design   google     4.6            59
447                     google   google     4.9          1.9k
448                     google   google     4.8           439


In [8]:
# Remove duplicates, keeping the one with highest rating, then most reviews
courses_clean = courses_clean.sort_values(['rating', 'reviews_count'], ascending=[False, False])
courses_clean = courses_clean.drop_duplicates('dedup_key', keep='first')
courses_clean = courses_clean.drop('dedup_key', axis=1)

print(f"After deduplication: {len(courses_clean)} courses")

After deduplication: 991 courses


## 4. Remove Dead Links

In [9]:
# Remove courses with invalid URLs
print(f"Before removing invalid URLs: {len(courses_clean)}")
courses_clean = courses_clean[courses_clean['valid_url']]
courses_clean = courses_clean.drop('valid_url', axis=1)
print(f"After removing invalid URLs: {len(courses_clean)}")

Before removing invalid URLs: 991
After removing invalid URLs: 991


## 5. Canonical Tag Generation

In [10]:
# Define canonical tag mapping
TAG_MAPPING = {
    # Programming Languages
    'python': ['python', 'django', 'flask', 'pandas', 'numpy', 'scikit-learn', 'tensorflow', 'pytorch'],
    'javascript': ['javascript', 'js', 'node.js', 'react', 'angular', 'vue', 'typescript'],
    'java': ['java', 'spring', 'hibernate'],
    'csharp': ['c#', '.net', 'asp.net'],
    'cpp': ['c++', 'cpp'],
    'r': ['r programming', 'rstudio'],
    'sql': ['sql', 'mysql', 'postgresql', 'database'],
    'go': ['golang', 'go programming'],
    'rust': ['rust programming'],
    'php': ['php', 'laravel'],
    
    # Domains
    'machine-learning': ['machine learning', 'ml', 'deep learning', 'neural network', 'ai', 'artificial intelligence'],
    'data-science': ['data science', 'data analysis', 'statistics', 'analytics'],
    'web-development': ['web development', 'frontend', 'backend', 'full stack', 'html', 'css'],
    'mobile-development': ['mobile', 'android', 'ios', 'react native', 'flutter'],
    'cloud': ['aws', 'azure', 'google cloud', 'cloud computing'],
    'cybersecurity': ['security', 'cybersecurity', 'ethical hacking', 'penetration testing'],
    'blockchain': ['blockchain', 'cryptocurrency', 'bitcoin', 'ethereum'],
    'devops': ['devops', 'docker', 'kubernetes', 'jenkins', 'ci/cd'],
    'business': ['business', 'management', 'marketing', 'finance', 'entrepreneurship'],
    'design': ['design', 'ui/ux', 'graphic design', 'photoshop'],
    'project-management': ['project management', 'agile', 'scrum'],
    'communication': ['communication', 'presentation', 'writing'],
    'leadership': ['leadership', 'team management'],
    'mathematics': ['mathematics', 'calculus', 'linear algebra', 'statistics'],
    'physics': ['physics', 'quantum'],
    'biology': ['biology', 'bioinformatics'],
    'economics': ['economics', 'microeconomics', 'macroeconomics'],
    'psychology': ['psychology', 'cognitive science'],
    'health': ['health', 'medicine', 'healthcare'],
    'education': ['education', 'teaching', 'pedagogy']
}

def extract_canonical_tags(title, description, original_tags):
    """Extract canonical tags from course content"""
    canonical_tags = set()
    
    # Combine text for analysis
    text = f"{title} {description}".lower()
    
    # Add original tags if they exist
    if pd.notna(original_tags) and original_tags:
        try:
            if isinstance(original_tags, str):
                # Try to parse as list
                if original_tags.startswith('['):
                    tags_list = ast.literal_eval(original_tags)
                else:
                    tags_list = [tag.strip() for tag in original_tags.split(',')]
                text += ' ' + ' '.join(str(tag).lower() for tag in tags_list)
        except:
            pass
    
    # Map keywords to canonical tags
    for canonical_tag, keywords in TAG_MAPPING.items():
        for keyword in keywords:
            if keyword in text:
                canonical_tags.add(canonical_tag)
                break
    
    return list(canonical_tags)

print("Canonical tag mapping defined")

Canonical tag mapping defined


In [11]:
# Generate canonical tags
print("Generating canonical tags...")

courses_clean['canonical_tags'] = courses_clean.apply(
    lambda row: extract_canonical_tags(row['title'], row['description'], row['tags']), 
    axis=1
)

# Count tag distribution
all_tags = []
for tags in courses_clean['canonical_tags']:
    all_tags.extend(tags)

tag_counts = pd.Series(all_tags).value_counts()
print(f"\nTop 10 canonical tags:")
print(tag_counts.head(10))

print(f"\nCourses with no tags: {(courses_clean['canonical_tags'].apply(len) == 0).sum()}")
print(f"Average tags per course: {courses_clean['canonical_tags'].apply(len).mean():.2f}")

Generating canonical tags...

Top 10 canonical tags:
machine-learning      740
business              512
design                376
data-science          251
communication         212
python                182
mobile-development    132
health                112
sql                   105
cloud                  97
Name: count, dtype: int64

Courses with no tags: 17
Average tags per course: 3.53


## 6. Final Data Preparation

In [12]:
# Select and reorder final columns
final_columns = [
    'title', 'description', 'provider', 'url', 'price', 'is_free', 
    'level', 'hours', 'rating', 'language', 'canonical_tags',
    'certificate_type', 'reviews_count', 'students_enrolled', 'summary'
]

# Ensure all columns exist
for col in final_columns:
    if col not in courses_clean.columns:
        courses_clean[col] = np.nan

courses_final = courses_clean[final_columns].copy()

# Convert tags to string format for parquet compatibility
courses_final['tags'] = courses_final['canonical_tags'].apply(lambda x: ','.join(x) if x else '')
courses_final = courses_final.drop('canonical_tags', axis=1)

# Clean numeric columns
courses_final['rating'] = pd.to_numeric(courses_final['rating'], errors='coerce')
courses_final['hours'] = pd.to_numeric(courses_final['hours'], errors='coerce')
courses_final['price'] = pd.to_numeric(courses_final['price'], errors='coerce')

# Parse enrollment numbers
def parse_enrollment(enrollment_str):
    if pd.isna(enrollment_str):
        return np.nan
    
    enrollment_str = str(enrollment_str).replace(',', '')
    numbers = re.findall(r'\d+', enrollment_str)
    return int(numbers[0]) if numbers else np.nan

courses_final['students_enrolled'] = courses_final['students_enrolled'].apply(parse_enrollment)
courses_final['reviews_count'] = pd.to_numeric(courses_final['reviews_count'], errors='coerce')

print(f"Final dataset shape: {courses_final.shape}")
print(f"Final columns: {list(courses_final.columns)}")

Final dataset shape: (991, 15)
Final columns: ['title', 'description', 'provider', 'url', 'price', 'is_free', 'level', 'hours', 'rating', 'language', 'certificate_type', 'reviews_count', 'students_enrolled', 'summary', 'tags']


In [13]:
# Display summary statistics
print("=== FINAL DATASET SUMMARY ===")
print(f"Total courses: {len(courses_final)}")
print(f"Unique providers: {courses_final['provider'].nunique()}")
print(f"Unique titles: {courses_final['title'].nunique()}")

print("\nLevel distribution:")
print(courses_final['level'].value_counts())

print("\nTop 10 providers:")
print(courses_final['provider'].value_counts().head(10))

print("\nRating statistics:")
print(courses_final['rating'].describe())

print("\nMissing values:")
print(courses_final.isnull().sum())

=== FINAL DATASET SUMMARY ===
Total courses: 991
Unique providers: 159
Unique titles: 987

Level distribution:
level
beginner        676
intermediate    199
unknown          80
advanced         36
Name: count, dtype: int64

Top 10 providers:
provider
google                                        87
ibm                                           78
university of pennsylvania                    55
deeplearning.ai                               43
google cloud                                  35
university of michigan                        34
university of illinois at urbana-champaign    29
university of colorado boulder                28
johns hopkins university                      26
duke university                               25
Name: count, dtype: int64

Rating statistics:
count    985.000000
mean       4.677665
std        0.194143
min        2.700000
25%        4.600000
50%        4.700000
75%        4.800000
max        5.000000
Name: rating, dtype: float64

Missing values:
title  

In [14]:
# Sample of cleaned data
print("Sample of cleaned data:")
courses_final.head()

Sample of cleaned data:


,title,description,provider,url,price,is_free,level,hours,rating,language,certificate_type,reviews_count,students_enrolled,summary,tags
465,healthier materials and sustainable building,in the healthier materials and sustainable bui...,"parsons school of design, the new school",https://www.coursera.org/specializations/healt...,0,True,intermediate,80.0,5.0,english,Specialization,9.0,NaN,"['Examine the human health, environmental, and...","design,machine-learning,health"
166,computational neuroscience,embark on an exciting journey into the world o...,university of washington,https://www.coursera.org/learn/computational-n...,0,True,beginner,80.0,5.0,english,Course,6.0,NaN,['Understand and perform basic security tasks ...,"design,cybersecurity,machine-learning"
401,generative ai with large language models,"instructed by ai pioneer andrew ng, generative...",deeplearning.ai,https://www.coursera.org/learn/generative-ai-w...,0,True,intermediate,25.0,5.0,english,Course,29.0,38552.0,"['What generative AI is and how it works, its ...","business,python,machine-learning"
139,calculus for machine learning and data science,the covid-19 crisis has created an unprecedent...,deeplearning.ai,https://www.coursera.org/learn/machine-learnin...,0,True,beginner,25.0,4.9,english,Course,NaN,1403637.0,"['Describe the natural history of SARS-CoV-2 ,...","mathematics,health,communication,data-science,..."
201,customer experiences with contact center ai - ...,"in this course, you will:\n• compare functiona...",google cloud,https://www.coursera.org/specializations/custo...,0,True,intermediate,180.0,4.9,english,Specialization,936.0,30099.0,[],"python,machine-learning"


## 7. Export to Parquet

In [15]:
# Create data directory if it doesn't exist
import os
os.makedirs('../data', exist_ok=True)

# Export to parquet
output_path = '../data/courses.parquet'
courses_final.to_parquet(output_path, index=False)

print(f"✅ Cleaned dataset exported to: {output_path}")
print(f"📊 Final dataset: {courses_final.shape[0]} courses, {courses_final.shape[1]} columns")

# Verify the export
verification = pd.read_parquet(output_path)
print(f"✅ Verification: Successfully loaded {len(verification)} courses from parquet file")

✅ Cleaned dataset exported to: ../data/courses.parquet
📊 Final dataset: 991 courses, 15 columns
✅ Verification: Successfully loaded 991 courses from parquet file
